# Trucks only problem #

Install necessary packages

In [20]:
from gurobipy import Model,GRB,LinExpr,quicksum
import numpy as np
from scipy.spatial import distance
import os
import socket
from load_dataset import Dataset

For Ugo's laptop

In [21]:
# Define the node name or another identifier of your laptop
my_laptop_node = 'Ugos-MacBook-Pro.local'

# Get the current system's node name using socket.gethostname()
current_node = socket.gethostname()

if current_node == my_laptop_node:
    # Set the environment variable for Gurobi license file
    os.environ["GRB_LICENSE_FILE"] = "/Users/ugomunzi/gurobi/licenses/gurobi.lic"
    print("Gurobi license path set for Ugo's MacBook Pro.")
else:
    print("Not Ugo's MacBook Pro, using default or no specific license settings.")

Not Ugo's MacBook Pro, using default or no specific license settings.


Define model parametres

In [22]:
## MODEL PARAMETERS ##
W_T = 1500 #empty weight truck [kg]
Q_T = 1000 #load capacity of trucks [kg]
W_D = 25 #empty weight drone [kg]
Q_D = 5 #load capacity of drones [kg]
C_T = 25 #travel cost of trucks per unit distance [monetary unit/km]
C_D = 1 #travel cost of drones per unit distance [monetary unit/km]
C_B = 500 #basis cost of using a truck equipped with a drone [monetary unit]
E = 0.5 #maximum endurance of empty drones [hours]
S_T = 60 #average travel speed of the trucks [km/h]
S_D = 65 #average travel speed of the drones [km/h]

Define Big M constant

In [23]:
M = 500 #big M constant for big M method

Load Dataset using load_dataset.py

In [24]:
## LOAD DATASET ##
current_dir = os.getcwd()
# Select which data folder to use
data_subfolder = '0.3'
data_subfoldercopy = '0.3_copy'
data_num_nodes = '40'
data_area = '20'

data_file_name = f'{data_num_nodes}_{data_area}_{data_subfoldercopy}'
dataset_path = f'dataset/{data_subfolder}/{data_file_name}.txt'
output_file_path = os.path.join(current_dir, data_file_name + '_solution.sol')#used to save solution file

dataset = Dataset(dataset_path)



Pre-processing

In [25]:
## FUNCTIONS ##
def get_manhattan_distance(data):
    """
    Returns a dictionary with manhattan distances between all nodes in dataset
    """
    distance_dict = {}
    for node1 in data.keys():
        for node2 in data.keys():
            distance_dict[node1, node2] = distance.cityblock([data[node1]['X'], data[node1]['Y']], [data[node2]['X'], data[node2]['Y']])
    return distance_dict

def get_time_dict(data, S_T, distance_dict):
    """
    Returns a dictionary with travel times between all nodes in dataset
    """
    time_dict = {}
    for node1 in data.keys():
        for node2 in data.keys():
            time_dict[node1, node2] = distance_dict[node1, node2] / S_T
    return time_dict


num_trucks = 2
distance_dict = get_manhattan_distance(dataset.data)
time_dict = get_time_dict(dataset.data, S_T, distance_dict)

#definitions of N_0, N and N_plus follow from paper
N = list(dataset.data.keys()) #set of nodes with depot at start
N_customers = N.copy()
N_customers.remove('D0')
V = [f'V{i}' for i in range(1, num_trucks+1)] #set of trucks

Create the model

In [26]:
## MODEL ##
model = Model("Truck-Only Model")

#decision variables
#define x such that you cannot travel between same node
x = model.addVars(V, [(i,j) for i in N for j in N if i != j], lb=0, ub=1, vtype=GRB.BINARY, name='x')
y = model.addVars(V, lb=0, ub=1, vtype=GRB.BINARY, name='y')
t = model.addVars(V, N, lb=0, vtype=GRB.CONTINUOUS, name='t')

model.update()

Set model parameters

In [27]:
model.setParam('TimeLimit', 600)  # Set to 10 minutes
model.setParam('MIPGap', 1e-4)  # Set the acceptable gap

Set parameter TimeLimit to value 600


Define the constraints

In [28]:
#constraints
# Each customer is visited by exactly one truck
model.addConstrs((quicksum(x[v, i, j] for v in V for j in N if i != j) == 1 for i in N_customers), name='Each_customer_visited_once')

# Each truck leaves the depot
model.addConstrs((quicksum(x[v, 'D0', j] for j in N_customers) == y[v] for v in V), name='Truck_leaves_depot')

# Each truck returns to the depot
model.addConstrs((quicksum(x[v, i, 'D0'] for i in N_customers) == y[v] for v in V), name='Truck_returns_depot')

# If a vechicle arrives at a customer node, it must leave the node
model.addConstrs((quicksum(x[v, i, j] for j in N if i != j) == quicksum(x[v, j, k] for j in N if j != k) for v in V for i in N_customers for k in N_customers if i != k), name='Flow')

# Time at the node is equal or larger than time at previous node
# plus travel time. Eliminates need for subtour constraints
model.addConstrs((t[v, j] >= t[v, i] + time_dict[i, j] - M*(1-x[v, i, j]) for v in V for i in N for j in N if i != j), name='Time_constraint')

# Payload for all visited customer nodes per vehicle is less than limit
#model.addConstrs((quicksum(dataset.data[j]['Demand']*x[v, i, j] for j in N_customers) <= Q_T for v in V for i in N_customers), name='Payload_limit')

{('V1', 'D0', 'C1'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C2'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C3'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C4'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C5'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C6'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C7'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C8'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C9'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'D0', 'C10'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'C1', 'D0'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'C1', 'C2'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'C1', 'C3'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'C1', 'C4'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'C1', 'C5'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1', 'C1', 'C6'): <gurobi.Constr *Awaiting Model Update*>,
 ('V1',

Run the optimiser

In [29]:
#objective function (minimise cost both due to tranportation and basis cost of using truck (if active, i.e. y=1))
cost_obj = quicksum(C_T * distance_dict[i,j] * x[v,i,j] for i in N for j in N if i != j for v in V) + quicksum(C_B * y[v] for v in V)
model.setObjective(cost_obj, GRB.MINIMIZE)
model.update()
model.write('TruckonlySimple.lp')
#tune solver before optimizing to reduce time it takes
#model.tune()
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license 2518537 - for non-commercial use only - registered to j.___@student.tudelft.nl
Optimize a model with 414 rows, 244 columns and 4144 nonzeros
Model fingerprint: 0xe3aa6742
Variable types: 22 continuous, 222 integer (222 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [5e+01, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Presolve time: 0.04s
Presolved: 414 rows, 244 columns, 4144 nonzeros
Variable types: 22 continuous, 222 integer (222 binary)

Root relaxation: objective 1.605000e+03, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

Post-processing

In [ ]:
## POST-PROCESSING ##
solution = {}
for var in model.getVars():
    solution[var.varName] = var.x

#exctract active vehicles
active_vehicles = [v for v in V if solution[f'y[{v}]'] >= 0.99]
#extract routes
active_routes = {}
for v in active_vehicles:
    active_routes[v] = [i for i in N if solution[f'x[{v},{i},D0]'] >= 0.99] #start with depot
    while active_routes[v][-1] != 'D0':
        for j in N:
            if solution[f'x[{v},{active_routes[v][-1]},{j}]'] >= 0.99:
                active_routes[v].append(j)
                break

#retrieve timestamps of customer visits
timestamps = {}
for v in active_vehicles:
    timestamps[v] = {}
    for i in N_customers:
        timestamps[v][i] = solution[f't[{v},{i}]']

#print all solution variables which have value of 1
print([var for var in solution.keys() if solution[var] >=0.9])
print(active_routes)
#plot routes
dataset.plot_data(show_demand=True, scale_nodes=True, show_labels=False, active_routes=active_routes)

AttributeError: Unable to retrieve attribute 'x'